# Dense Neural Network (NN) Exploration

In this notebook, we will lay out our approach for a Neural Network approach to predict a pose based on landmark. Additionally, we are storing out angle calculation and perspectively our angle error calculation.

In [1]:
import numpy as np
import pandas as pd

## Neural Network

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [3]:
# X = data.drop(columns = 'label')
# y = 

In [4]:
# X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size = 0.3)

In [5]:
# def initialize_model():
    
#     #############################
#     #  1 - Model architecture   #
#     ############################# 
    
#     model = models.Sequential()
#     model.add(layers.Dense(20, activation='relu', input_dim=51))
#     model.add(layers.Dense(10, activation='relu', input_dim=51))
#     model.add(layers.Dense(10, activation='relu', input_dim=51))
#     model.add(layers.Dense(5, activation='softmax'))
#     # $CHALLENGIFY_END
    
#     #############################
#     #  2 - Optimization Method  #
#     #############################
#     model.compile(loss='categorical_crossentropy', # different from binary_crossentropy because we have multiple classes
#                   optimizer='adam', 
#                   metrics=['accuracy']) 

#     return model 


# model = initialize_model()

In [6]:
# model.summary()

In [7]:
# model = initialize_model()

# history = model.fit(X_train, y_train, 
#                     epochs=50, 
#                     batch_size=16, 
#                     verbose=0)

In [8]:
# def plot_history(history):
#     plt.plot(history.history['loss'])
#     plt.title('Model loss')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.show()

# plot_history(history)

In [9]:
# results = model.evaluate(X_test, y_test, verbose=0)
# print(results[1])

## Masaki

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2
import os
import pandas as pd

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display


In [11]:
# Model Initialization
model_name = "movenet_lightning"
module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
input_size = 192
model = module.signatures['serving_default']

In [66]:
# Choose image file and preprocess

image_path = "../clean/00000134.jpg"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)


In [67]:
# Detect landmarks on image
model = module.signatures["serving_default"]
input_image = tf.cast(input_image, dtype=tf.int32)
input_image = input_image[..., :3]
outputs = model(input_image)

# 1D array of xyz coordinates. X1, Y1, Z1, X2, ... , Yn, Zn format
xyz_individual = outputs["output_0"].numpy()


In [72]:
best_pose = xyz_individual

In [14]:
# Helper method for drawing landmarks
def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot

In [15]:
# Get list of image file names in given directory
dir_path_train_dd = "../clean"
file_names = []
for entry in os.listdir(dir_path_train_dd):
    file_names.append(entry)

In [62]:
# Create DF of xyz values
df = []
image_names = []

for file in file_names:
    image_names.append(dir_path_train_dd)
    dir_path_train_dd = f"../clean/{file}"
    if not file.endswith((".jpg", ".png")):
        pass
    else:
#       print(dir_path_train_dd)
        image = tf.io.read_file(dir_path_train_dd)
        image = tf.image.decode_jpeg(image)
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
        input_image = input_image[..., :3]

        model = module.signatures["serving_default"]
        input_image = tf.cast(input_image, dtype=tf.int32)
        outputs = model(input_image)
        df.append(outputs["output_0"].numpy().reshape(51).tolist())


In [63]:
data = pd.DataFrame(df)

In [18]:
data["label"] = "downdog"

In [19]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,label
0,0.614686,0.366069,0.722570,0.612122,0.350986,0.643456,0.612995,0.350938,0.715975,0.563880,...,0.515043,0.705209,0.887617,0.685450,0.883067,0.797601,0.667293,0.864593,0.652384,downdog
1,0.534696,0.722514,0.564541,0.529799,0.733964,0.405342,0.526650,0.732507,0.399033,0.498358,...,0.471535,0.375810,0.738871,0.586280,0.311128,0.580621,0.628086,0.273356,0.892581,downdog
2,0.569520,0.529188,0.600391,0.572470,0.513902,0.517100,0.575602,0.515343,0.508116,0.547930,...,0.502984,0.773414,0.708213,0.674067,0.870783,0.614912,0.665915,0.863403,0.627864,downdog
3,0.621766,0.494117,0.670643,0.623084,0.508379,0.587127,0.625575,0.502040,0.504647,0.590072,...,0.494093,0.241079,0.877534,0.604555,0.187687,0.531326,0.631976,0.162686,0.756716,downdog
4,0.615906,0.424954,0.598805,0.618176,0.415418,0.541572,0.616567,0.410814,0.479552,0.595441,...,0.533408,0.626742,0.754802,0.652928,0.721405,0.676844,0.650494,0.716107,0.753108,downdog


In [55]:
# Create DF of xyz values
df = []
xyz = []
for file in file_names:
    dir_path_train_dd = f"../clean/{file}"
    if not file.endswith((".jpg", ".png")):
        pass
    else:
        print(dir_path_train_dd)
        image = tf.io.read_file(dir_path_train_dd)
        image = tf.image.decode_jpeg(image)
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
        input_image = input_image[..., :3]

        model = module.signatures["serving_default"]
        input_image = tf.cast(input_image, dtype=tf.int32)
        outputs = model(input_image)
        df.append(outputs["output_0"].numpy().reshape(51).tolist())
        xyz.append(outputs["output_0"].numpy()[0][0])

## Angle Calculator

In [21]:
image_landmarks = xyz[6][:, :-1]
image_landmarks

array([[0.66960835, 0.35614344],
       [0.67016554, 0.33723804],
       [0.6718497 , 0.33988258],
       [0.6360858 , 0.30929214],
       [0.6374579 , 0.31444016],
       [0.58631724, 0.3324954 ],
       [0.58074266, 0.331631  ],
       [0.70875835, 0.27349722],
       [0.6967046 , 0.26761028],
       [0.7763366 , 0.1773637 ],
       [0.75954664, 0.19102556],
       [0.40240896, 0.53494716],
       [0.40276787, 0.5312333 ],
       [0.56703186, 0.63760805],
       [0.57365   , 0.6361871 ],
       [0.7234893 , 0.7441256 ],
       [0.7093338 , 0.7298353 ]], dtype=float32)

In [22]:
lm = image_landmarks
lm

array([[0.66960835, 0.35614344],
       [0.67016554, 0.33723804],
       [0.6718497 , 0.33988258],
       [0.6360858 , 0.30929214],
       [0.6374579 , 0.31444016],
       [0.58631724, 0.3324954 ],
       [0.58074266, 0.331631  ],
       [0.70875835, 0.27349722],
       [0.6967046 , 0.26761028],
       [0.7763366 , 0.1773637 ],
       [0.75954664, 0.19102556],
       [0.40240896, 0.53494716],
       [0.40276787, 0.5312333 ],
       [0.56703186, 0.63760805],
       [0.57365   , 0.6361871 ],
       [0.7234893 , 0.7441256 ],
       [0.7093338 , 0.7298353 ]], dtype=float32)

In [23]:
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

In [24]:
# landmark_dict = {
#     'landmarks_left_elbow' : (lm[5], lm[7], lm[9]),
#     'landmarks_right_elbow' : (lm[6], lm[8], lm[10]),
#     'landmarks_left_shoulder' : (lm[7], lm[9], lm[11]),
#     'landmarks_right_shoulder' : (lm[8], lm[10], lm[12]),
#     'landmarks_hip_left' : (lm[5], lm[11], lm[13]),
#     'landmarks_hip_right' : (lm[6], lm[12], lm[14]),
#     'landmarks_left_knee' : (lm[11], lm[13], lm[15]),
#     'landmarks_right_knee' : (lm[12], lm[14], lm[16])
#         }

In [25]:
def angle_function(lm):
    
    landmark_dict = {
    'landmarks_left_elbow' : (lm[5], lm[7], lm[9]),
    'landmarks_right_elbow' : (lm[6], lm[8], lm[10]),
    'landmarks_left_shoulder' : (lm[7], lm[5], lm[11]),
    'landmarks_right_shoulder' : (lm[8], lm[6], lm[12]),
    'landmarks_hip_left' : (lm[5], lm[11], lm[13]),
    'landmarks_hip_right' : (lm[6], lm[12], lm[14]),
    'landmarks_left_knee' : (lm[11], lm[13], lm[15]),
    'landmarks_right_knee' : (lm[12], lm[14], lm[16])
        }
    
    def angle_calculator(landmarks):

        # Converting the points into numpy arrays
        p1 = np.array(landmarks[0])
        p2 = np.array(landmarks[1])
        p3 = np.array(landmarks[2])

        # Creating the Vectors between two points
        vec_p1 = p1-p2
        vec_p2 = p3-p2

        # Calculating the cosine of the angle
        cosine_angle = np.dot(vec_p1,vec_p2) / (np.linalg.norm(vec_p1)*np.linalg.norm(vec_p2))
        
        #clipping cosine angle
        cosine_angle = np.clip(cosine_angle, -1, 1)
        # Calculating angle
        angle = np.arccos(cosine_angle)

        # Calculating degrees
        angle_degrees = np.degrees(angle)

        return angle_degrees

    angle_name_list = ['angle_elbow_left',
                   'angle_elbow_right',
                   'angle_shoulder_left',
                   'angle_shoulder_right',
                   'angle_hip_left',
                   'angle_hip_right',
                   'angle_knee_left',
                   'angle_knee_right',
                  ]

    angle_dict = {}
    
    for dictkey, listkey in zip(landmark_dict, angle_name_list):
        i = angle_calculator(landmark_dict[dictkey])
        angle_dict[listkey] = i
    
    angle_list = [angle_dict[key] for key in angle_dict]
    
    return angle_dict, angle_list
    
    
    
    

In [162]:
angle_dict = angle_function(lm)[0]

In [27]:
angle_list = angle_function(lm)[1]

In [163]:
angle_dict

{'angle_elbow_left': 150.83283888944322,
 'angle_elbow_right': 158.27316727244093,
 'angle_shoulder_left': 157.97924378229874,
 'angle_shoulder_right': 160.62407016602953,
 'angle_hip_left': 79.69599921076946,
 'angle_hip_right': 79.83605384090288,
 'angle_knee_left': 177.7007495861799,
 'angle_knee_right': 176.9444989140278}

In [ ]:
###### angle_name_list = ['angle_elbow_left',
#                    'angle_elbow_right',
#                    'angle_shoulder_left',
#                    'angle_shoulder_right',
#                    'angle_hip_left',
#                    'angle_hip_right',
#                    'angle_knee_left',
#                    'angle_knee_right',
#                   ]

In [29]:
# angle_dict = {}

In [30]:
# for dictkey, listkey in zip(landmark_dict, angle_name_list):
#     i = angle_calculator(landmark_dict[dictkey])
#     angle_dict[listkey] = i

In [31]:
angle_dict

{'angle_elbow_left': 150.83283888944322,
 'angle_elbow_right': 158.27316727244093,
 'angle_shoulder_left': 157.97924378229874,
 'angle_shoulder_right': 160.62407016602953,
 'angle_hip_left': 79.69599921076946,
 'angle_hip_right': 79.83605384090288,
 'angle_knee_left': 177.7007495861799,
 'angle_knee_right': 176.9444989140278}

These are the angles we need!

In [32]:
# angle_list = [angle_dict[key] for key in angle_dict]

In [33]:
angle_list

[150.83283888944322,
 158.27316727244093,
 157.97924378229874,
 160.62407016602953,
 79.69599921076946,
 79.83605384090288,
 177.7007495861799,
 176.9444989140278]

In [34]:
xyz

[array([[0.61468613, 0.36606914, 0.7225705 ],
        [0.61212206, 0.35098642, 0.6434561 ],
        [0.6129949 , 0.3509376 , 0.7159747 ],
        [0.56388015, 0.3286766 , 0.58742386],
        [0.5627878 , 0.3274661 , 0.5724267 ],
        [0.5053564 , 0.37220737, 0.75928754],
        [0.5080349 , 0.37157354, 0.7369339 ],
        [0.6347408 , 0.26611096, 0.7730443 ],
        [0.6133088 , 0.26804513, 0.77417135],
        [0.70369303, 0.15653855, 0.71411836],
        [0.675892  , 0.1728544 , 0.58133334],
        [0.26079118, 0.6013876 , 0.8471483 ],
        [0.26600903, 0.60989666, 0.8646514 ],
        [0.5165872 , 0.7039327 , 0.8456954 ],
        [0.51504314, 0.70520854, 0.8876169 ],
        [0.6854501 , 0.8830673 , 0.7976013 ],
        [0.667293  , 0.86459327, 0.6523842 ]], dtype=float32),
 array([[0.5346956 , 0.7225141 , 0.56454104],
        [0.5297986 , 0.73396367, 0.40534174],
        [0.5266502 , 0.73250693, 0.39903274],
        [0.4983579 , 0.71041316, 0.46751082],
        [0.491044

In [35]:
lst = []
lst

[]

## Angle Comparison Tool

In [36]:
lst = []
for x in xyz:
    lst.append(angle_function(x[:, :-1])[1])
    
    

In [37]:
image_names

['../clean',
 '../clean/00000372.jpg',
 '../clean/00000414.jpg',
 '../clean/00000158.jpg',
 '../clean/00000164.jpg',
 '../clean/00000170.jpg',
 '../clean/00000366.png',
 '../clean/00000206.jpg',
 '../clean/00000213.jpg',
 '../clean/00000165.jpg',
 '../clean/00000159.jpg',
 '../clean/00000198.png',
 '../clean/00000417.jpg',
 '../clean/00000403.jpg',
 '../clean/00000173.jpg',
 '../clean/00000239.jpg',
 '../clean/00000210.jpg',
 '../clean/00000204.jpg',
 '../clean/00000238.jpg',
 '../clean/00000199.jpg',
 '../clean/00000166.jpg',
 '../clean/00000172.jpg',
 '../clean/00000416.jpg',
 '../clean/00000358.jpg',
 '../clean/00000412.jpg',
 '../clean/00000406.jpg',
 '../clean/00000360.jpg',
 '../clean/00000176.jpg',
 '../clean/00000162.jpg',
 '../clean/00000189.jpg',
 '../clean/00000228.png',
 '../clean/00000200.jpg',
 '../clean/.DS_Store',
 '../clean/00000214.jpg',
 '../clean/00000229.jpg',
 '../clean/00000201.jpg',
 '../clean/00000349.png',
 '../clean/00000163.jpg',
 '../clean/00000375.jpg',
 '

In [38]:
lst

[[161.53362240213735,
  167.843976509054,
  176.2120896618278,
  179.97202354488172,
  64.98513327120168,
  65.50150346889245,
  155.15446269197722,
  154.6316161278501],
 [156.64711088827488,
  161.68898203335698,
  172.80663412212004,
  170.62885210026198,
  76.42771559177098,
  76.1864240755983,
  178.6312638965127,
  177.90846844439045],
 [163.1606397340297,
  168.44315808116878,
  163.9766233636938,
  173.6246478969662,
  83.535028104302,
  79.59260058764433,
  174.3220754127887,
  172.93597455601034],
 [159.1311227641873,
  162.4533006323687,
  175.0713272176276,
  176.93822657154527,
  67.05004095241361,
  66.10043484338323,
  170.02058470259522,
  173.78858061752678],
 [161.46189654942748,
  159.2727566475548,
  178.72835246866262,
  179.94404708809589,
  63.241979090460326,
  62.17602401948884,
  172.6173802757785,
  170.07512327567514],
 [176.88888008193132,
  179.36541823337794,
  167.14676872368796,
  171.5335730392964,
  71.4553943659471,
  68.85792628085264,
  174.7678195

In [39]:
array = np.asarray(lst)

In [40]:
df = pd.DataFrame(lst)

In [41]:
angle_name_list = ['angle_elbow_left',
                   'angle_elbow_right',
                   'angle_shoulder_left',
                   'angle_shoulder_right',
                   'angle_hip_left',
                   'angle_hip_right',
                   'angle_knee_left',
                   'angle_knee_right',
                  ]

In [42]:
df.columns = angle_name_list

In [43]:
len(image_names)

174

In [44]:
index_image_names = pd.Index(image_names)[1:]

In [45]:
index_image_names

Index(['../clean/00000372.jpg', '../clean/00000414.jpg',
       '../clean/00000158.jpg', '../clean/00000164.jpg',
       '../clean/00000170.jpg', '../clean/00000366.png',
       '../clean/00000206.jpg', '../clean/00000213.jpg',
       '../clean/00000165.jpg', '../clean/00000159.jpg',
       ...
       '../clean/00000154.jpg', '../clean/00000183.jpg',
       '../clean/00000197.jpg', '../clean/00000222.jpg',
       '../clean/00000237.jpg', '../clean/00000155.jpg',
       '../clean/00000394.jpg', '../clean/00000419.jpg',
       '../clean/00000357.jpg', '../clean/00000343.jpg'],
      dtype='object', length=173)

In [46]:
df

,angle_elbow_left,angle_elbow_right,angle_shoulder_left,angle_shoulder_right,angle_hip_left,angle_hip_right,angle_knee_left,angle_knee_right
0,161.533622,167.843977,176.212090,179.972024,64.985133,65.501503,155.154463,154.631616
1,156.647111,161.688982,172.806634,170.628852,76.427716,76.186424,178.631264,177.908468
2,163.160640,168.443158,163.976623,173.624648,83.535028,79.592601,174.322075,172.935975
3,159.131123,162.453301,175.071327,176.938227,67.050041,66.100435,170.020585,173.788581
4,161.461897,159.272757,178.728352,179.944047,63.241979,62.176024,172.617380,170.075123
...,...,...,...,...,...,...,...,...
168,179.701293,165.721554,175.804693,179.078040,76.873131,78.148940,170.602358,177.309354
169,161.681059,149.090703,179.039170,171.703812,82.515012,85.690439,171.275595,170.211822
170,171.262838,170.055190,167.761186,172.788547,70.871124,71.581266,174.285500,174.015681
171,160.956188,163.733047,174.155778,177.717238,79.114631,79.048406,179.692259,179.955765


In [47]:
df.set_index(index_image_names, inplace=True)

In [48]:
df.to_csv('/Users/lennartjanssen/code/lennijanssen/angles_test')

## Angle Error & Score

In [168]:
#Finding values for the best pose

# Choose image file and preprocess

image_path = "../clean/00000144.jpg"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)


# Detect landmarks on image
model = module.signatures["serving_default"]
input_image = tf.cast(input_image, dtype=tf.int32)
input_image = input_image[..., :3]
outputs = model(input_image)

# 1D array of xyz coordinates. X1, Y1, Z1, X2, ... , Yn, Zn format
best_pose = outputs["output_0"].numpy()[:, :, :, :-1][0][0]

In [169]:
#Finding values for the test pose

# Choose image file and preprocess

image_path = "../clean/00000338.jpg"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)


# Detect landmarks on image
model = module.signatures["serving_default"]
input_image = tf.cast(input_image, dtype=tf.int32)
input_image = input_image[..., :3]
outputs = model(input_image)

# 1D array of xyz coordinates. X1, Y1, Z1, X2, ... , Yn, Zn format
test_pose = outputs["output_0"].numpy()[:, :, :, :-1][0][0]

In [170]:
best_pose[0]

array([0.6573974 , 0.63276094], dtype=float32)

In [171]:
angle_function(best_pose)

({'angle_elbow_left': 179.48186703999414,
  'angle_elbow_right': 171.86500944196104,
  'angle_shoulder_left': 174.92082243673866,
  'angle_shoulder_right': 177.67695262265346,
  'angle_hip_left': 73.04021647596142,
  'angle_hip_right': 71.51305474999457,
  'angle_knee_left': 177.9711669605012,
  'angle_knee_right': 179.96573597990397},
 [179.48186703999414,
  171.86500944196104,
  174.92082243673866,
  177.67695262265346,
  73.04021647596142,
  71.51305474999457,
  177.9711669605012,
  179.96573597990397])

In [184]:
from operator import truediv

def angle_comparer(test_pose, best_pose):
    
    best_pose_angles = angle_function(best_pose)[1]
    test_pose_angles = angle_function(test_pose)[1]
    
    
    print(best_pose_angles)
    print(test_pose_angles)
    
    angle_substraction = [a - b for a, b in zip(best_pose_angles, test_pose_angles)]
    
    
    angle_difference = list(map(abs, angle_substraction))
    
    
    test_angle_percentage = list(map(truediv, best_pose_angles, test_pose_angles))
    
    
    test_angle_percentage_diff = [round(abs(x-1),2) for x in test_angle_percentage]
    
    
    average_percentage_diff = round(sum(test_angle_percentage_diff)/len(test_angle_percentage_diff),2)
    
    
    return test_angle_percentage_diff, average_percentage_diff
    

In [185]:
angle_comparer(test_pose,best_pose)

[179.48186703999414, 171.86500944196104, 174.92082243673866, 177.67695262265346, 73.04021647596142, 71.51305474999457, 177.9711669605012, 179.96573597990397]
[177.09835485889602, 167.98403520158175, 151.8588520158713, 152.36791613238526, 69.21435712076618, 80.58657037127409, 167.40068394964456, 174.6690583361485]


([0.01, 0.02, 0.15, 0.17, 0.06, 0.11, 0.06, 0.03], 0.08)